## Fashion MNIST Non Balanced Dataset with Bigger Model

In [1]:
import torch
import torchvision

import numpy as np
import math

In [2]:
mnist_trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5,), (0.5,))
                             ]))
mnist_testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5,), (0.5,))
                             ]))

In [3]:
## Parameters:
n_epochs = 3
batch_size_train = 10000
batch_size_test = 500
log_interval = 500

In [4]:
train_loader = torch.utils.data.DataLoader(mnist_trainset,batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset,batch_size=batch_size_test, shuffle=False)

In [5]:
import papayaclient

In [6]:
class TheModel(torch.nn.Module):

    def __init__(self):
        super(TheModel, self).__init__()

        self.conv1 = torch.nn.Conv2d(1, 1, 5)
        self.linear1 = torch.nn.Linear((24 * 24), 10)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = x1.flatten(start_dim = 1)
        return self.linear1(x2)

In [7]:
clients = []
list_of_data = []
list_of_labels = []
for batchno, (ex_data, ex_labels) in enumerate(train_loader):
    list_of_data.append(ex_data)
    list_of_labels.append(ex_labels)
data, labels = torch.cat(list_of_data), torch.cat(list_of_labels)
np.random.seed(42)
idx = [0] + sorted(np.random.choice(59999, 5, replace=False)+1) + [60000]
for i in range(6):
    ex_data = data[idx[i]:idx[i+1]]
    ex_labels = labels[idx[i]:idx[i+1]]
    clients.append(papayaclient.PapayaClient(dat = ex_data,
                                        labs = ex_labels,
                                        batch_sz = 500,
                                        num_partners = 5,
                                        model_class = TheModel,
                                        loss_fn = torch.nn.CrossEntropyLoss))

In [8]:
## Train the Nodes
num_epochs_total = 100
num_epochs_per_swap = 5
num_times = (num_epochs_total // num_epochs_per_swap)
for i in range(0, num_times):
    for n in clients:
        for j in range(0, num_epochs_per_swap):
            n.model_train_epoch()
    if i > 1 and i < num_times - 1 :
        for n in clients:
            n.select_partners(3)
        for n in clients:
            for i in range(0, 4) :
                n.update_partner_weights()
            n.average_partners()

In [9]:
for c in clients :
    print(c.logs['stringy'][99])

node1299epoch 99 loss 0.43982821702957153
node450epoch 99 loss 0.3593287467956543
node1028epoch 99 loss 0.3897861838340759
node4568epoch 99 loss 0.37207454442977905
node54epoch 99 loss 0.4991750717163086
node667epoch 99 loss 0.39443331956863403


In [10]:
accuracies = {}
with torch.no_grad():
    for i in clients :
        accuracies_node = []
        for batchno, (ex_data, ex_labels) in enumerate(test_loader) :
            accuracies_node.append(((i.model.forward(ex_data).argmax(dim = 1) == ex_labels).float().mean()).item())
        accuracies[i.node_id] = np.array(accuracies_node).mean()

In [11]:
accuracies

{1299: 0.7964000076055526,
 450: 0.8146000027656555,
 1028: 0.8029000043869019,
 4568: 0.8259999990463257,
 54: 0.8180999994277954,
 667: 0.808899998664856}